In [1]:
###############################################################################
# 0) SETUP (unchanged)
###############################################################################
%load_ext autoreload
%autoreload 2
import asyncio

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources.hamilton import (
    STARLetDeck, MFX_CAR_L5_base, TIP_CAR_480_A00
)
from pylabrobot.resources.hamilton.mfx_modules import MFX_DWP_module_188042
from pylabrobot.resources import (
    HTF,             # 300 µL filtered tips
    TIP_50ul_w_filter,        # 50 µL filtered tips
    Cor_96_wellplate_360ul_Fb
)



In [2]:

###############################################################################
# 1) BUILD LH + DECK
###############################################################################
backend = STARBackend()
lh = LiquidHandler(backend=backend, deck=STARLetDeck())
await lh.setup(skip_autoload=True)        # faster while iterating

In [3]:
from typing import Optional

from pylabrobot.resources.height_volume_functions import (
  compute_height_from_volume_rectangle,
  compute_volume_from_height_rectangle,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)

def BioER_96_wellplate_Vb_2200ul(name: str, lid: Optional[Lid] = None) -> Plate:
  """
  BioER plate similar to NEST and other KingFisher-compatible deep well plates
  https://en.bioer.com/uploadfiles/2024/05/20240513165756879.pdf
  Cat no. BSH06M1T-A
  """
  INNER_WELL_WIDTH = 8.2  # from spec  
  INNER_WELL_LENGTH = 8.2  # from spec

  well_kwargs = {
    "size_x": INNER_WELL_WIDTH,  # measured
    "size_y": INNER_WELL_LENGTH,  # measured
    "size_z": 42.4,  # from spec
    "bottom_type": WellBottomType.V,
    "cross_section_type": CrossSectionType.RECTANGLE,
    "compute_height_from_volume": lambda liquid_volume: compute_height_from_volume_rectangle(
      liquid_volume,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    "compute_volume_from_height": lambda liquid_height: compute_volume_from_height_rectangle(
      liquid_height,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    # "material_z_thickness": 1,
    "material_z_thickness": 0.80, # measured
  }

  return Plate(
    name=name,
    size_x=127.30,  # from spec
    size_y=85.20,  # from spec
    size_z=44.2,  # from spec
    lid=lid,
    model=BioER_96_wellplate_Vb_2200ul.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=9.2,  # measured
      dy=9.5,  # measured
      dz=1.2,  # from spec
      item_dx=9, # from spec
      item_dy=9, # from spec
      **well_kwargs,
    ),
  )

In [4]:

##############################################################################
#2) CARRIERS & LABWARE (unchanged rails; only tip rows differ)
##############################################################################
tip_car        = TIP_CAR_480_A00("tip_car")
tip_car[0]     = HTF(name="htf_tips")       # 300 µL
tip_car[1]     = TIP_50ul_w_filter(name="htf_50ul")  # 50 µL
lh.deck.assign_child_resource(tip_car, rails=1)

src_mod        = MFX_DWP_module_188042("src_mod")
src_car        = MFX_CAR_L5_base("src_car", modules={0: src_mod})
lh.deck.assign_child_resource(src_car, rails=13)
src_plate      = BioER_96_wellplate_Vb_2200ul("source_plate")
src_mod.assign_child_resource(src_plate, location=0)

# destination (flat-bottom) carrier on rail 19
dst_mod        = MFX_DWP_module_188042("dst_mod")
dst_car        = MFX_CAR_L5_base("dst_car", modules={0: dst_mod})
lh.deck.assign_child_resource(dst_car, rails=19)
dst_plate      = Cor_96_wellplate_360ul_Fb("dest_plate")
dst_mod.assign_child_resource(dst_plate, location=0)


In [5]:
###############################################################################
# 3) ACCURACY-OPT SETTINGS  (all 8 channels)
###############################################################################
precise = dict(
    aspirate_flow_rate = 10,
    dispense_flow_rate = 10,
    end_delay_seconds  = 0.5,
    settling_time      = [0.5]*8
)

lld_surface = dict(
    lld_mode = [STARBackend.LLDMode.GAMMA]*8,
)

lld_surface_follow = lld_surface | {
    "surface_following_distance": [10]*8
}

###############################################################################
# 4) HELPERS
###############################################################################
ROWS      = "ABCDEFGH"          # 8 rows → 8 channels
USE_CHANS = list(range(8))

def first_well(item):
    return item[0] if isinstance(item, (list, tuple)) else item

###############################################################################
# 5) PICK UP 300 µL HTF TIPS & PRE-WET
###############################################################################
src_plate_instance = lh.deck.get_resource('source_plate')
# rack tips
tip_rack = lh.deck.get_resource("htf_tips")
await lh.pick_up_tips(tip_rack["A12:H12"], use_channels=USE_CHANS)
# prime tips
await lh.aspirate(src_plate_instance["A12:H12"], vols=[300]*8, use_channels=USE_CHANS, **precise, **lld_surface_follow)
await lh.dispense(src_plate_instance["A12:H12"], vols=[300]*8, use_channels=USE_CHANS, **precise, blow_out=[1]*8, liquid_height=[35]*8)

###############################################################################
# 6) WATER-VOLUME ACCURACY  (199 µL, 190 µL, 100 µL)
###############################################################################
water_src_11 = [first_well(src_plate[f"{row}11"]) for row in ROWS]  # A11-H11
water_src_12 = [first_well(src_plate[f"{row}12"]) for row in ROWS]  # A12-H12

def pick_water_source(col: int):
    """Return the correct source-well list for a given destination column."""
    return water_src_11 if col <= 6 else water_src_12

water_tests = [
    (range(1, 5),  199),   # cols 1-4
    (range(5, 9),  190),   # cols 5-8
    (range(9, 12), 180),   # cols 9-11
]

for cols, vol in water_tests:
    for col in cols:
        dst_wells = [first_well(dst_plate[f"{row}{col}"]) for row in ROWS]
        src_wells = pick_water_source(col)
        # await lh.aspirate(src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, **lld_surface_follow)
        try:
            await lh.aspirate(src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, **lld_surface_follow)
        except:
            print ("Channels woudl have errored here.")
            await lh.aspirate(src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[1]*8)
        await lh.dispense(dst_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[3]*8)   # no blow-out

###############################################################################
# 7) DISCARD 300 µL TIPS & PICK UP 50 µL TIPS
###############################################################################
await lh.discard_tips()

tip_car = lh.deck.get_resource("htf_50ul")
await lh.pick_up_tips(tip_car["A6:H6"], use_channels=USE_CHANS)
###############################################################################
# 8) ORANGE G TEST (1 µL, 10 µL, 20 µL)
###############################################################################
dye_src_wells = [first_well(src_plate[f"{row}1"]) for row in ROWS]
dye_lld       = lld_surface | {"immersion_depth": [1]*8}

dye_tests = [
    (range(1, 5),   1),    # cols 1-4
    (range(5, 9),  10),    # cols 5-8
    (range(9, 12), 20),    # cols 9-11
]

for cols, vol in dye_tests:
    for col in cols:
        dst_wells = [first_well(dst_plate[f"{row}{col}"]) for row in ROWS]
        await lh.aspirate(dye_src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, **dye_lld)
        await lh.dispense(dst_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[5]*8)   # no blow-out

###############################################################################
# 9) FINAL DISCARD
###############################################################################
await lh.discard_tips()


/home/hamilton-robot/Documents/Hamilton-Starlet/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:332: UserWarning: Extra arguments to backend.aspirate: {'dispense_flow_rate', 'end_delay_seconds', 'aspirate_flow_rate'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")
/home/hamilton-robot/Documents/Hamilton-Starlet/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:332: UserWarning: Extra arguments to backend.dispense: {'dispense_flow_rate', 'end_delay_seconds', 'aspirate_flow_rate'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")


In [6]:
# await lh.dispense(src, vols=[50]*8, use_channels=[0], blow_out=[1])
# await lh.blow_out(location=waste["A1"], use_channels=channels)
# await lh.dispense(src_wells, vols=[100]*8, use_channels=USE_CHANS, **precise, liquid_height=[30]*8)   # no blow-out
# # tip_rack = lh.deck.get_resource("htf_tips")
# await lh.drop_tips(tip_rack["A12:H12"], use_channels=list(range(8)))
# # await lh.drop_tips(tiprack["A1:C1"])
# # await lh.discard_tips()
# # await lh.prepare_for_manual_channel_operation()
await lh.stop()